## Introduction

We analyze Japanese stock market from Yahoo! Finance. In this notebook, let's cluster data from correlation matrix.

## Exploratory Analysis

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


There is 1 csv file in the current version of the dataset:


In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


The next hidden code cells define functions for plotting data. Click on the "Code" button in the published kernel to reveal the hidden code.

In [ ]:
# Distribution graphs (histogram/bar graph) of column data
def plotPerColumnDistribution(df, nGraphShown, nGraphPerRow):
    nunique = df.nunique()
    df = df[[col for col in df if nunique[col] > 1 and nunique[col] < 50]] # For displaying purposes, pick columns that have between 1 and 50 unique values
    nRow, nCol = df.shape
    columnNames = list(df)
    nGraphRow = (nCol + nGraphPerRow - 1) / nGraphPerRow
    plt.figure(num = None, figsize = (6 * nGraphPerRow, 8 * nGraphRow), dpi = 80, facecolor = 'w', edgecolor = 'k')
    for i in range(min(nCol, nGraphShown)):
        plt.subplot(nGraphRow, nGraphPerRow, i + 1)
        columnDf = df.iloc[:, i]
        if (not np.issubdtype(type(columnDf.iloc[0]), np.number)):
            valueCounts = columnDf.value_counts()
            valueCounts.plot.bar()
        else:
            columnDf.hist()
        plt.ylabel('counts')
        plt.xticks(rotation = 90)
        plt.title(f'{columnNames[i]} (column {i})')
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()


In [ ]:
# Correlation matrix
def plotCorrelationMatrix(df, graphWidth):
    filename = df.dataframeName
    df = df.dropna('columns') # drop columns with NaN
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    if df.shape[1] < 2:
        print(f'No correlation plots shown: The number of non-NaN or constant columns ({df.shape[1]}) is less than 2')
        return
    corr = df.corr()
    plt.figure(num=None, figsize=(graphWidth, graphWidth), dpi=80, facecolor='w', edgecolor='k')
    corrMat = plt.matshow(corr, fignum = 1)
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.columns)), corr.columns)
    plt.gca().xaxis.tick_bottom()
    plt.colorbar(corrMat)
    plt.title(f'Correlation Matrix for {filename}', fontsize=15)
    plt.show()


In [ ]:
# Scatter and density plots
def plotScatterMatrix(df, plotSize, textSize):
    df = df.select_dtypes(include =[np.number]) # keep only numerical columns
    # Remove rows and columns that would lead to df being singular
    df = df.dropna('columns')
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    columnNames = list(df)
    if len(columnNames) > 10: # reduce the number of columns for matrix inversion of kernel density plots
        columnNames = columnNames[:10]
    df = df[columnNames]
    ax = pd.plotting.scatter_matrix(df, alpha=0.75, figsize=[plotSize, plotSize], diagonal='kde')
    corrs = df.corr().values
    for i, j in zip(*plt.np.triu_indices_from(ax, k = 1)):
        ax[i, j].annotate('Corr. coef = %.3f' % corrs[i, j], (0.8, 0.2), xycoords='axes fraction', ha='center', va='center', size=textSize)
    plt.suptitle('Scatter and Density Plot')
    plt.show()


Now you're ready to read in the data and use the plotting functions to visualize the data.

### Let's check 1st file: /kaggle/input/all_japanese_stocks.csv

In [ ]:
df1 = pd.read_csv('/kaggle/input/all_japanese_stocks.csv', delimiter=',')
df1.dataframeName = 'all_japanese_stocks.csv'
columns = df1.columns.to_list()
columns[0] = "code"
df1.columns = columns
df1["Date"] = pd.to_datetime(df1["Date"])
nRow, nCol = df1.shape
print(f'There are {nRow} rows and {nCol} columns')

Let's take a quick look at what the data looks like:

In [ ]:
df1.head(5)

In [ ]:
df = df1.set_index(["code", "Date"])
df = df[(df["Volume"] > 0) & (df["Close"] < df["Close"].rolling(10).mean()*2)]
df["TurnOver"] = df["Close"]*df["Volume"]
print(df.shape)
df.head(5)

In [ ]:
turnover = df[["TurnOver"]].unstack().T
turnover.index = turnover.index.get_level_values(1)
turnover = turnover.sort_index()
turnover = turnover.dropna(thresh=2000, axis=1).ffill().dropna()
turnover.head(5)

In [ ]:
target_stocks = turnover.mean().nlargest(1000, keep='last')
target_stocks.index

In [ ]:
close_price = df[["Close"]].unstack().T
close_price.index = close_price.index.get_level_values(1)
close_price = close_price.sort_index()
close_price = close_price.dropna(thresh=2000, axis=1).ffill().dropna()
first_date = close_price.index[0]
close_price = close_price[target_stocks.index]
close_price.head(5)

# Clustering

So let's cluster stocks here.

Be careful that $\mbox{corr}(X, Y)$ is not a distance of $X$ and $Y$.

Let's define a distance metrics as $d(X, Y) = \sqrt{1-\mbox{corr}(X, Y)^2}$.

In [ ]:
corr = close_price.pct_change().corr()
corr.head()

In [ ]:
distance_corr = np.sqrt(1-corr**2)
distance_corr.head()

In [ ]:
import seaborn as sns

sns.clustermap(distance_corr.values[:100][:, :100])

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(round(distance_corr[distance_corr.columns[:30]].iloc[:30],2), cmap='RdBu', annot=True, 
            annot_kws={"size": 7}, vmin=-1, vmax=1);

As we can see the histgram of distance in the dataset, almost all pairs of stocks are irrelevant.

However, let's see pairs of stocks which have a close distance.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.hist(np.ravel(distance_corr[distance_corr > 0]), density=True, bins=100)
ax.axvline(0.5, ymin=0, ymax=1, color="red", linestyle="dashed")
ax.grid(True)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.hist(np.ravel(distance_corr[(distance_corr > 0) & (distance_corr < 0.5)]), density=True, bins=100)
ax.grid(True)

In [ ]:
pairs = distance_corr[(distance_corr > 0) & (distance_corr < 0.2)].apply(pd.Series.last_valid_index).dropna().reset_index()
pairs

In [ ]:
fig, ax = plt.subplots(nrows=len(pairs),figsize=(15, 2*len(pairs)))
k = 0
for stock in pairs.values:
    for n in range(2):
        _stock = df.loc[stock[n]]["Close"][first_date:]
        ax[k].plot(_stock/(_stock.iloc[0]), label=stock[n])
    ax[k].set_title("correlation : {:.2f}".format(corr.loc[stock[0]][stock[1]]))
    ax[k].grid(True)
    ax[k].legend()
    k += 1
plt.tight_layout()

## Conclusion

We are doing EDA here and try to pick up some close pairs of stocks in this notebook.

As we can see the last graph, results might be helpful to find pairs of stocks.

In this case, we only discover ETF with same underlying, however we can explore further to see if there are unregular pairs of individual stocks.

Good luck!